In [1]:
from datetime import date, timedelta
import win32com.client
from win32com.client import Dispatch, constants
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.files.file import File 
import io
import pandas as pd
import json

In [2]:
f = open(r'T:\DCProjects\GitHub\LCOG_support\config\keys.json')
crd = json.load(f)
f.close

<function TextIOWrapper.close()>

In [3]:
url = 'https://lanecouncilofgovernments.sharepoint.com/:x:/r/sites/LCOGDiversityEquityBelonging/_layouts/15/Doc.aspx?sourcedoc=%7B95B76D95-363F-452E-A99D-77524AA7A68E%7D&file=Facilitator%20and%20Notetaker%20Rotation.xlsx&action=default&mobileredirect=true'

In [4]:
username, password = crd['SharePoint']['usr'], crd['SharePoint']['pwd']

In [5]:
ctx_auth = AuthenticationContext(url)
if ctx_auth.acquire_token_for_user(username, password):
    ctx = ClientContext(url, ctx_auth)
    web = ctx.web
    ctx.load(web)
    ctx.execute_query()
    print('Authentication Successful')
    print(web.properties)
else:
    print(ctx_auth.get_last_error())

Authentication Successful
{}


In [6]:
response = File.open_binary(ctx, url)

In [7]:
#save data to BytesIO stream
bytes_file_obj = io.BytesIO()
bytes_file_obj.write(response.content)
bytes_file_obj.seek(0) #set file object to start

0

In [17]:
#read file into pandas dataframe
#df = pd.read_excel(bytes_file_obj)

In [11]:
sitename = 'LCOGDiversityEquityBelonging'

In [12]:
# Credential to connect to your SP Site
SITE_URL =f'https://lanecouncilofgovernments.sharepoint.com/sites/{sitename}'
CLIENT_ID, CLIENT_SECRET = crd['SharePoint']['cid'], crd['SharePoint']['scr']

In [13]:
context = ClientContext(SITE_URL).with_credentials(ClientCredential(CLIENT_ID, CLIENT_SECRET))

In [14]:
response = File.open_binary(context, '/'.join([f'/sites/{sitename}/Shared Documents/Committee Meetings/Agendas', 
                                           'Facilitator and Notetaker Rotation.xlsx']))

ValueError: {"error":"unauthorized_client","error_description":"AADSTS700016: Application with identifier '1c7b4efb-8eec-4ccc-bda2-f6fd0940709f' was not found in the directory 'lanecouncilofgovernments'. This can happen if the application has not been installed by the administrator of the tenant or consented to by any user in the tenant. You may have sent your authentication request to the wrong tenant.\r\nTrace ID: 1091ee03-24ae-4ad8-bf79-28c890110300\r\nCorrelation ID: 0dc3ac6e-6e59-41f3-b2b4-3adbfcd602f9\r\nTimestamp: 2023-02-09 20:37:28Z","error_codes":[700016],"timestamp":"2023-02-09 20:37:28Z","trace_id":"1091ee03-24ae-4ad8-bf79-28c890110300","correlation_id":"0dc3ac6e-6e59-41f3-b2b4-3adbfcd602f9","error_uri":"https://accounts.accesscontrol.windows.net/error?code=700016"}

In [ ]:
df = pd.read_excel(io.BytesIO(response.content))

In [ ]:
def get_sharepoint_context_using_user():
 
    # Get sharepoint credentials
    sharepoint_url = 'https://lanecouncilofgovernments.sharepoint.com'

    # Initialize the client credentials
    user_credentials = UserCredential(crd['SharePoint']['usr'], crd['SharePoint']['pwd'])

    # create client context object
    ctx = ClientContext(sharepoint_url).with_credentials(user_credentials)

    return ctx

In [ ]:
def get_files(file_url):
    try:

        site_url = "https://lanecouncilofgovernments.sharepoint.com"
        credentials = ClientCredential(crd['SharePoint']['usr'], crd['SharePoint']['pwd'])

        ctx = ClientContext(site_url).with_credentials(credentials)
        
        # file_url is the sharepoint url from which you need the list of files
        list_source = ctx.web.get_folder_by_server_relative_url(file_url)
        files = list_source.files
        ctx.load(files)
        ctx.execute_query()

        return files

    except Exception as e:
        print(e)

In [ ]:
get_files(url)

In [ ]:
import datetime

def third_Wednesday(year, month):
    """Return datetime.date for monthly option expiration given year and
    month
    """
    # The 15th is the lowest third day in the month
    third = datetime.date(year, month, 15)
    # What day of the week is the 15th?
    w = third.weekday()
    # Wednesday is weekday 2
    if w != 2:
        # Replace just the day (of month)
        third = third.replace(day=(15 + (2 - w) % 7))
    return third

In [ ]:
date1 = third_Wednesday(2023, 2)

In [ ]:
date1

In [ ]:
date2 = date.today() + timedelta(days=7)

In [ ]:
const=win32com.client.constants
olMailItem = 0x0
obj = win32com.client.Dispatch("Outlook.Application")
newMail = obj.CreateItem(olMailItem)
newMail.Subject = f'Reminder: DEIB meeting next Wednesday {str(date1)}'
newMail.BodyFormat = 2 # olFormatHTML https://msdn.microsoft.com/en-us/library/office/aa219371(v=office.11).aspx
newMail.HTMLBody =  """
<HTML><BODY><p>Hello all,</p> 

<p>Our meeting facilitator and notetaker next week are Denise and Kate, and next month are Ellen and Heidi. Following the links, please find the folders for the <a href="https://lanecouncilofgovernments.sharepoint.com/:f:/r/sites/LCOGDiversityEquityBelonging/Shared%20Documents/Committee%20Meetings/Agendas/2023?csf=1&web=1&e=wBl76p">meeting agenda</a>, and <a href="https://lanecouncilofgovernments.sharepoint.com/:f:/r/sites/LCOGDiversityEquityBelonging/Shared%20Documents/Committee%20Meetings/Meeting%20Notes/2023?csf=1&web=1&e=03DAbf">meeting notes</a>.

Meeting facilitators will need to fill out the agenda details. Please note that we will have a hybrid meeting this time, and the in-person meeting room is McKenzie Room on PPB 4th floor.</p> 

<p>Have a great day!</p>

<p>Dongmei on behalf of DEIB Strategic Planning Subcommittee</p></BODY></HTML>
"""
newMail.To = "LCOGDiversityEquityInclusionBelonging@lcog.org"
#newMail.From = "LCOGDiversityEquityInclusionBelonging@lcog.org"

#newMail.Send()